In [2]:
# Import the various modules
%run ImportYupNotebook.ipynb
import glob
import os

import csv

In [3]:
#path information common to all the code
OrPath = 'H:/My Drive/2023_CCVRUSA_LSC/Analysis/ThresholdScan/'
# load the run db
rundata = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/RunList.csv')
Runs = rundata[rundata.Include==1].Run.to_numpy()
RunsDAQ = rundata[rundata.Include==1]['RunCrossDAQ'].to_numpy()
Runs = [950035]# Runs[np.argsort(RunsDAQ)]


In [4]:
#channel names/ codes
# detector mapping
DetMap = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/ChannelsList.csv')

Channels = DetMap.Channel.to_numpy()
Names = DetMap.Name.to_numpy()
CHNameMap = {Channels[i]: Names[i] for i in range(len(Channels))}

AvailChannels = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]#[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]#Channels[2:-2]
AvailLDs = [n for n in AvailChannels if DetMap.Sensor[DetMap.Channel==n].to_numpy()[0]=="Light" ]
AvailHDs = [n for n in AvailChannels if DetMap.Sensor[DetMap.Channel==n].to_numpy()[0]=="Heat" ]


# get current THs from SB

In [5]:
sb = {}
sb[950023] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/20230302_sbOptDebAvgTrig_100mHz_ModCH12.csv').sort_values('id')
sb[950024] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/20230302_sbOptDebAvgTrig_100mHz_ModCH12.csv').sort_values('id')
sb[950029] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/sb79.csv').sort_values('id')
sb[950033] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/Run740_FromSB81_NewTHs.csv').sort_values('id')

sb[950037] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/sb83.csv').sort_values('id')
sb[950035] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/sb86.csv').sort_values('id')
sb[950040] = pd.read_csv('H:/My Drive/2023_CCVRUSA_LSC/Analysis/SettingsBasket/sb83.csv').sort_values('id')

# Get Scans from TXT

In [6]:
#list of files
#reading and saving the output into a dataframe
column_names = ["Run", "Channel", "THs","Rates"]

AllScans = pd.DataFrame(columns = column_names)


#reading and saving the output into a dataframe
for run in Runs:
  for ch in [12,16]: #AvailHDs:#AvailChannels:
    # file to be read
    # file to be read
    
    f = OrPath+'/run{:d}_Deb60/ScanTHs_run{:d}_CH{:d}.txt'.format(run,run,ch)

    #read the scan
    my_data = pd.read_csv(f)
    ths = my_data.Threshold.to_numpy()
    rate = my_data.Rate.to_numpy()
    
    #filling the df
    el = [run,ch,ths,rate] 
    ss = pd.DataFrame([el],columns = column_names)
    AllScans = AllScans.append(ss);


# Draw the Rate vs Threshold plots
comparing each channel on the different configurations

In [14]:
cmap = plt.cm.get_cmap('jet')
for ch in [12]:#]AvailHDs: #[20]:# [10]:#
  fig, ax = plt.subplots(1)  
  #plot the current SB threshold
  currth = sb[run][sb[run].id==ch]['deriv. trg threshold'].values[0]            
  ax.axvspan(currth-0.005, currth+0.005, facecolor='r', alpha=1,label = 'Current TH')
  for it,run in enumerate(Runs):
    
    selsp = np.logical_and(np.equal(AllScans.Run.to_numpy(),run) , np.equal(AllScans.Channel,ch))

    if np.sum(selsp)>0:
        ths = AllScans[selsp].THs.values[0]
        rates = AllScans[selsp].Rates.values[0]
        #plot
        ax.errorbar(ths,rates,yerr=0,xerr=0,
                              color = cmap(it/np.size(Runs)),
                              marker='o', fmt = '',linestyle='',label="Run {:d} ".format(run),ms = 4)

        
        
        # ax.axvspan(currth, currth, facecolor='r', alpha=1,label = 'Current TH')
        ax.set( yscale='log',
                xscale='log',
                xlabel = 'Threshold',
                ylabel = 'Rate [Hz]',
                title = f'Channel {ch} ' + CHNameMap[ch],
                xlim = [np.min(ths),np.max(ths)])#np.max(freqs)])np.min(ths)
  ax.legend(loc='best',prop={'size': 6})
  
  # ax.figure.savefig(OrPath+r'/Plots/ANPS_AP_Comparison_CH{:d}.png'.format(ch),dpi=600)  
#   ax.figure.savefig(OrPath+r'/Plot/ANPS_Comparison_CH{:d}.png'.format(ch),dpi=600)  

Checkignt the db scan for CH 12

In [7]:
Averages=[2,6,10,20,40,60]
Debounces = [2,4,6,8,10]

MarkerDebounce = {}
MarkerDebounce[2] = 'o'
MarkerDebounce[4] = '+'
MarkerDebounce[6] = 'v'
MarkerDebounce[8] = 's'
MarkerDebounce[10] = '^'

ColAvgs = {}
ColAvgs[100] = 'tab:blue'
ColAvgs[80] = 'tab:orange'
ColAvgs[60] = 'tab:green'
ColAvgs[40] = 'tab:red'
ColAvgs[20] = 'tab:purple'
ColAvgs[10] = 'tab:gray'
ColAvgs[6] = 'tab:pink'
ColAvgs[2] = 'turquoise'

OrPathScan = 'H:/My Drive/2023_CCVRUSA_LSC/Analysis/ThresholdScan/TrigScan_MultiSpace/'

In [11]:
column_names = ["Run", "Channel", "THs","Rates","Average","Debounce"]

AllScans = pd.DataFrame(columns = column_names)
for ch in [12,14,16,18,20]:
  for av in Averages:
    for db in Debounces:
      f = OrPathScan+f'Avg{av:.0f}/Deb{db:.0f}/run950035/ScanTHs_run950035_CH{ch}.txt'

      #read the scan
      my_data = pd.read_csv(f)
      ths = my_data.Threshold.to_numpy()
      rate = my_data.Rate.to_numpy()
      
      #filling the df
      el = [950035,ch,ths,rate,av,db] 
      ss = pd.DataFrame([el],columns = column_names)
      AllScans = AllScans.append(ss);

In [12]:
plotsavg = {av: plt.subplots(1) for av in Averages}
plotsdb = {db: plt.subplots(1) for db in Debounces}
fig, ax = plt.subplots(1)  
cmap = plt.cm.get_cmap('jet')
ch = 14



for av in Averages:
  for db in Debounces:
  
    selsp = np.logical_and.reduce((np.equal(AllScans.Debounce.to_numpy(),db), 
                                   np.equal(AllScans.Average.to_numpy(),av), 
                                   np.equal(AllScans.Channel,ch)))

    if np.sum(selsp)>0:
      ths = AllScans[selsp].THs.values[0]
      rates = AllScans[selsp].Rates.values[0]
      #plot
      ax.errorbar(ths*av,rates,yerr=0,xerr=0,color = ColAvgs[av],
                            marker=MarkerDebounce[db], fmt = '',linestyle='',label=f"Avg {av} - Deb {db} ",ms = 4)
      plotsavg[av][0].axes[0].errorbar(ths*av,rates,yerr=0,xerr=0,color = ColAvgs[av],
                            marker=MarkerDebounce[db], fmt = '',linestyle='',label=f"Avg {av} - Deb {db} ",ms = 4)
      plotsdb[db][0].axes[0].errorbar(ths*av,rates,yerr=0,xerr=0,color = ColAvgs[av],
                            marker=MarkerDebounce[db], fmt = '',linestyle='',label=f"Avg {av} - Deb {db} ",ms = 4)
    
    
        
ax.set( yscale='log',
        xscale='log',
        xlabel = 'Threshold',
        ylabel = 'Rate [Hz]',
        title = f'Channel {ch} ' + CHNameMap[ch])
ax.legend(loc='best',prop={'size': 6})

for av in plotsavg:
  
  plotsavg[av][0].axes[0].set( yscale='log',
          xscale='log',
          xlabel = 'Threshold',
          ylabel = 'Rate [Hz]',
          title = f'Channel {ch} ' + CHNameMap[ch])
  plotsavg[av][0].axes[0].legend(loc='best',prop={'size': 6})
for db in plotsdb:
  
  plotsdb[db][0].axes[0].set( yscale='log',
          xscale='log',
          xlabel = 'Threshold',
          ylabel = 'Rate [Hz]',
          title = f'Channel {ch} ' + CHNameMap[ch])
  plotsdb[db][0].axes[0].legend(loc='best',prop={'size': 6})